In [1]:
import re
import warnings
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

In [2]:
from scipy import stats

In [3]:
file_path = r"F:\neuron project with Jesus"

df = pd.read_csv(f"{file_path}/imputed_raw.csv", index_col = 0) 

In [4]:
df_log = np.log2(df)
df_log

,CL2_VEH_3d_1,CL2_VEH_3d_2,CL2_VEH_3d_3,CL2_VEH_3d_4,CL2_VEH_3d_5,CL2_VEH_3d_6,CL5_VEH_3d_1,CL5_VEH_3d_2,CL5_VEH_3d_3,CL5_VEH_3d_4,...,CL2_DAPT_16d_3,CL2_DAPT_16d_4,CL2_DAPT_16d_5,CL2_DAPT_16d_6,CL5_DAPT_16d_1,CL5_DAPT_16d_2,CL5_DAPT_16d_3,CL5_DAPT_16d_4,CL5_DAPT_16d_5,CL5_DAPT_16d_6
Genes,,,,,,,,,,,,,,,,,,,,,
NUDT4B,19.035578,19.448651,19.766385,20.082037,19.357294,19.985542,18.484874,18.948684,18.637318,18.802699,...,18.713477,19.047449,18.972806,18.975742,19.038222,19.134797,19.211366,19.161850,19.260805,19.037308
PIGBOS1,14.951699,17.362783,15.367043,15.366537,16.029175,15.990944,17.281921,15.240788,15.979778,17.328193,...,15.765268,15.816101,14.878520,15.548591,14.892239,16.179244,15.309544,15.522915,15.445519,16.013419
TMEM275,16.048515,14.961748,16.605995,15.490757,15.696256,15.287240,16.843136,16.437775,16.988363,17.191223,...,14.742720,15.807667,16.215929,15.944637,16.081119,16.074501,15.831411,16.613213,15.735746,15.812227
CENPVL1,14.135084,14.051226,12.868041,14.656816,14.129806,14.102427,15.759386,15.349205,15.724346,16.025938,...,12.780029,13.993593,13.206637,15.095038,15.465755,15.659486,14.617284,15.027189,14.835922,14.458214
NBDY,19.863672,19.378401,19.480826,19.160425,19.356020,19.207412,19.526675,19.285217,19.271443,19.419339,...,18.451948,19.162581,18.274433,19.080877,18.790838,19.204179,18.310586,18.486265,18.327289,19.089833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MYO16,16.443802,16.416491,16.185718,16.370010,16.685775,15.834365,16.671030,16.110595,16.578623,16.311009,...,16.976531,16.746869,17.017352,16.595189,16.909014,17.156310,16.535426,16.528279,16.889100,17.040011
MORC2,20.852197,20.471824,20.796410,20.647321,20.793841,20.676866,20.939625,20.737612,20.856322,20.653212,...,21.285344,21.273549,21.261398,21.323160,20.853508,20.817908,20.751253,20.904989,20.859890,20.911418
IVNS1ABP,19.005331,19.000393,19.116024,19.208684,19.609408,19.427546,19.073467,19.156515,18.699596,18.974569,...,18.172344,18.417473,18.706563,18.647525,18.821160,19.018211,18.834665,18.859241,18.994019,18.659330


In [8]:
wide = df.copy()  # rows=proteins/genes, cols=samples

# ===== 2) 从列名解析 metadata =====
# CL2 / CL5, VEH / DAPT, 3d/4d/8d/16d, replicate 1-6
pat = re.compile(r"^(CL\d+)_(VEH|DAPT)_(\d+)d_(\d+)$")

meta = []
for c in wide.columns:
    m = pat.match(c)
    if not m:
        raise ValueError(f"Column name not match pattern: {c}")
    cell_line, drug, day, rep = m.group(1), m.group(2), int(m.group(3)), int(m.group(4))
    meta.append((c, cell_line, drug, day, rep))

meta_df = pd.DataFrame(meta, columns=["sample", "cell_line", "drug", "day", "rep"])

# ===== 3) 宽表 -> 长表 =====
long = (
    wide.reset_index()
    .melt(id_vars="Genes", var_name="sample", value_name="intensity")
    .merge(meta_df, on="sample", how="left")
)
long

,Genes,sample,intensity,cell_line,drug,day,rep
0,NUDT4B,CL2_VEH_3d_1,537378.0,CL2,VEH,3,1
1,PIGBOS1,CL2_VEH_3d_1,31689.1,CL2,VEH,3,1
2,TMEM275,CL2_VEH_3d_1,67777.3,CL2,VEH,3,1
3,CENPVL1,CL2_VEH_3d_1,17992.2,CL2,VEH,3,1
4,NBDY,CL2_VEH_3d_1,954028.0,CL2,VEH,3,1
...,...,...,...,...,...,...,...
831530,MYO16,CL5_DAPT_16d_6,134758.0,CL5,DAPT,16,6
831531,MORC2,CL5_DAPT_16d_6,1972260.0,CL5,DAPT,16,6
831532,IVNS1ABP,CL5_DAPT_16d_6,414017.0,CL5,DAPT,16,6
831533,CAMTA1,CL5_DAPT_16d_6,32835.1,CL5,DAPT,16,6


In [10]:
results = []

for gene, sub in long.groupby("Genes", sort=False):
    d = sub.dropna(subset=["intensity"]).copy()
    if d.shape[0] < 10:
        continue

    d["log2_intensity"] = np.log2(d["intensity"] + 1)

    # 固定 reference level（决定系数含义：相对于哪个baseline）
    d["cell_line"] = pd.Categorical(d["cell_line"], categories=["CL2", "CL5"])
    d["drug"]      = pd.Categorical(d["drug"],      categories=["VEH", "DAPT"])
    d["day"]       = pd.Categorical(d["day"],       categories=[3, 4, 8, 16], ordered=True)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = smf.ols("log2_intensity ~ C(cell_line) + C(drug) + C(day)", data=d).fit()

    # ===== A) 因子整体显著性：Type II ANOVA (F-test) =====
    aov = sm.stats.anova_lm(model, typ=2)

    # ===== B) 单个系数统计量：coef / std err / t / p / CI =====
    coef_tbl = model.summary2().tables[1].copy()  # index=term, columns include Coef., Std.Err., t, P>|t|
    ci = model.conf_int(alpha=0.05)               # 95% CI
    ci.columns = ["CI2.5%", "CI97.5%"]
    coef_tbl = coef_tbl.join(ci)

    # 我们关心的系数名字（如果你的分类水平变了，下面名字也会跟着变）
    term_map = {
        "coef_drug_DAPT_vs_VEH": "C(drug)[T.DAPT]",
        "coef_cell_CL5_vs_CL2":  "C(cell_line)[T.CL5]",
        "coef_day_4_vs_3":       "C(day)[T.4]",
        "coef_day_8_vs_3":       "C(day)[T.8]",
        "coef_day_16_vs_3":      "C(day)[T.16]",
    }

    row = {
        "Genes": gene,
        "n_obs": int(d.shape[0]),
        "r2": float(model.rsquared),

        # 因子整体F-test P值（你原来那三个）
        "p_cell_line": float(aov.loc["C(cell_line)", "PR(>F)"]),
        "p_drug":      float(aov.loc["C(drug)",      "PR(>F)"]),
        "p_day":       float(aov.loc["C(day)",       "PR(>F)"]),
    }

    # 安全取单个系数（有些蛋白可能因为共线性/缺少水平而缺term）
    def add_term_stats(prefix, term):
        if term in coef_tbl.index:
            row[prefix] = float(coef_tbl.loc[term, "Coef."])
            row[prefix.replace("coef_", "se_")] = float(coef_tbl.loc[term, "Std.Err."])
            row[prefix.replace("coef_", "t_")]  = float(coef_tbl.loc[term, "t"])
            row[prefix.replace("coef_", "pcoef_")] = float(coef_tbl.loc[term, "P>|t|"])
            row[prefix.replace("coef_", "ci2.5_")]  = float(coef_tbl.loc[term, "CI2.5%"])
            row[prefix.replace("coef_", "ci97.5_")] = float(coef_tbl.loc[term, "CI97.5%"])
        else:
            row[prefix] = np.nan
            row[prefix.replace("coef_", "se_")] = np.nan
            row[prefix.replace("coef_", "t_")]  = np.nan
            row[prefix.replace("coef_", "pcoef_")] = np.nan
            row[prefix.replace("coef_", "ci2.5_")]  = np.nan
            row[prefix.replace("coef_", "ci97.5_")] = np.nan

    for out_name, term in term_map.items():
        add_term_stats(out_name, term)

    results.append(row)

res = pd.DataFrame(results)
res

,Genes,n_obs,r2,p_cell_line,p_drug,p_day,coef_drug_DAPT_vs_VEH,se_drug_DAPT_vs_VEH,t_drug_DAPT_vs_VEH,pcoef_drug_DAPT_vs_VEH,...,t_day_8_vs_3,pcoef_day_8_vs_3,ci2.5_day_8_vs_3,ci97.5_day_8_vs_3,coef_day_16_vs_3,se_day_16_vs_3,t_day_16_vs_3,pcoef_day_16_vs_3,ci2.5_day_16_vs_3,ci97.5_day_16_vs_3
0,NUDT4B,95,0.131678,2.727828e-02,0.126352,1.061714e-01,-0.091369,0.059211,-1.543111,0.126352,...,0.161931,0.871727,-0.153661,0.180928,-0.006151,0.083275,-0.073866,9.412824e-01,-0.171617,0.159315
1,PIGBOS1,95,0.145017,3.396103e-01,0.588080,4.427435e-03,0.065144,0.119840,0.543594,0.588080,...,-3.657160,0.000431,-0.961806,-0.284612,-0.367890,0.168545,-2.182732,3.168817e-02,-0.702786,-0.032993
2,TMEM275,95,0.604732,2.954456e-15,0.004475,1.315702e-06,0.313994,0.107649,2.916825,0.004475,...,-3.375855,0.001092,-0.820905,-0.212600,-0.723636,0.151400,-4.779629,6.893209e-06,-1.024465,-0.422807
3,CENPVL1,95,0.622337,2.147679e-18,0.606685,8.725119e-05,-0.054085,0.104685,-0.516646,0.606685,...,-2.795054,0.006356,-0.711843,-0.120288,-0.419606,0.147231,-2.849986,5.432085e-03,-0.712151,-0.127061
4,NBDY,95,0.305526,1.302424e-03,0.295129,2.557654e-05,-0.117894,0.111945,-1.053137,0.295129,...,-5.157103,0.000002,-1.137211,-0.504628,-0.363284,0.157443,-2.307406,2.335206e-02,-0.676119,-0.050449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,MYO16,95,0.149731,1.571155e-01,0.025235,4.427061e-02,0.139129,0.061124,2.276171,0.025235,...,2.000065,0.048542,0.001137,0.346539,0.117304,0.085966,1.364533,1.758402e-01,-0.053509,0.288117
8748,MORC2,95,0.518826,3.192115e-11,0.406145,8.006606e-07,0.027446,0.032883,0.834658,0.406145,...,5.183535,0.000001,0.149467,0.335284,0.243435,0.046248,5.263729,9.686282e-07,0.151542,0.335328
8749,IVNS1ABP,95,0.471815,1.248813e-07,0.613518,3.701662e-08,-0.029279,0.057767,-0.506844,0.613518,...,-3.232453,0.001722,-0.428740,-0.102307,-0.243121,0.081245,-2.992431,3.581154e-03,-0.404554,-0.081688
8750,CAMTA1,95,0.145245,9.787987e-03,0.419735,7.044511e-02,0.067661,0.083466,0.810639,0.419735,...,2.657434,0.009333,0.079573,0.551223,0.195441,0.117388,1.664911,9.944718e-02,-0.037807,0.428688


In [11]:

# # ===== 4) baseline model: log2(intensity+1) ~ CellLine + Drug + Day =====
# # 输出每个蛋白对每个因子的整体P值（Type II ANOVA: PR(>F)）
# results = []

# for gene, sub in long.groupby("Genes", sort=False):
#     d = sub.dropna(subset=["intensity"]).copy()
#     if d.shape[0] < 10:  # 太少就跳过（你可以自己调整阈值）
#         continue

#     d["log2_intensity"] = np.log2(d["intensity"] + 1)

#     # 设定reference level（可选，但建议固定，便于解释系数）
#     d["cell_line"] = pd.Categorical(d["cell_line"], categories=["CL2", "CL5"])
#     d["drug"]      = pd.Categorical(d["drug"],      categories=["VEH", "DAPT"])
#     d["day"]       = pd.Categorical(d["day"],       categories=[3, 4, 8, 16], ordered=True)

#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         model = smf.ols("log2_intensity ~ C(cell_line) + C(drug) + C(day)", data=d).fit()

#     aov = sm.stats.anova_lm(model, typ=2)  # Type II ANOVA

#     row = {
#         "Genes": gene,
#         "n_obs": d.shape[0],
#         "r2": model.rsquared,
#         "p_cell_line": aov.loc["C(cell_line)", "PR(>F)"],
#         "p_drug":      aov.loc["C(drug)",      "PR(>F)"],
#         "p_day":       aov.loc["C(day)",       "PR(>F)"],
#     }
#     results.append(row)

# res = pd.DataFrame(results)

res

,Genes,n_obs,r2,p_cell_line,p_drug,p_day,coef_drug_DAPT_vs_VEH,se_drug_DAPT_vs_VEH,t_drug_DAPT_vs_VEH,pcoef_drug_DAPT_vs_VEH,...,t_day_8_vs_3,pcoef_day_8_vs_3,ci2.5_day_8_vs_3,ci97.5_day_8_vs_3,coef_day_16_vs_3,se_day_16_vs_3,t_day_16_vs_3,pcoef_day_16_vs_3,ci2.5_day_16_vs_3,ci97.5_day_16_vs_3
0,NUDT4B,95,0.131678,2.727828e-02,0.126352,1.061714e-01,-0.091369,0.059211,-1.543111,0.126352,...,0.161931,0.871727,-0.153661,0.180928,-0.006151,0.083275,-0.073866,9.412824e-01,-0.171617,0.159315
1,PIGBOS1,95,0.145017,3.396103e-01,0.588080,4.427435e-03,0.065144,0.119840,0.543594,0.588080,...,-3.657160,0.000431,-0.961806,-0.284612,-0.367890,0.168545,-2.182732,3.168817e-02,-0.702786,-0.032993
2,TMEM275,95,0.604732,2.954456e-15,0.004475,1.315702e-06,0.313994,0.107649,2.916825,0.004475,...,-3.375855,0.001092,-0.820905,-0.212600,-0.723636,0.151400,-4.779629,6.893209e-06,-1.024465,-0.422807
3,CENPVL1,95,0.622337,2.147679e-18,0.606685,8.725119e-05,-0.054085,0.104685,-0.516646,0.606685,...,-2.795054,0.006356,-0.711843,-0.120288,-0.419606,0.147231,-2.849986,5.432085e-03,-0.712151,-0.127061
4,NBDY,95,0.305526,1.302424e-03,0.295129,2.557654e-05,-0.117894,0.111945,-1.053137,0.295129,...,-5.157103,0.000002,-1.137211,-0.504628,-0.363284,0.157443,-2.307406,2.335206e-02,-0.676119,-0.050449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,MYO16,95,0.149731,1.571155e-01,0.025235,4.427061e-02,0.139129,0.061124,2.276171,0.025235,...,2.000065,0.048542,0.001137,0.346539,0.117304,0.085966,1.364533,1.758402e-01,-0.053509,0.288117
8748,MORC2,95,0.518826,3.192115e-11,0.406145,8.006606e-07,0.027446,0.032883,0.834658,0.406145,...,5.183535,0.000001,0.149467,0.335284,0.243435,0.046248,5.263729,9.686282e-07,0.151542,0.335328
8749,IVNS1ABP,95,0.471815,1.248813e-07,0.613518,3.701662e-08,-0.029279,0.057767,-0.506844,0.613518,...,-3.232453,0.001722,-0.428740,-0.102307,-0.243121,0.081245,-2.992431,3.581154e-03,-0.404554,-0.081688
8750,CAMTA1,95,0.145245,9.787987e-03,0.419735,7.044511e-02,0.067661,0.083466,0.810639,0.419735,...,2.657434,0.009333,0.079573,0.551223,0.195441,0.117388,1.664911,9.944718e-02,-0.037807,0.428688


In [13]:
# ===== C) 对“因子整体P值”做 BH-FDR（q值）=====
for pcol in ["p_cell_line", "p_drug", "p_day"]:
    p = res[pcol].values
    ok = np.isfinite(p)
    q = np.full_like(p, np.nan, dtype=float)
    if ok.sum() > 0:
        _, q_ok, _, _ = multipletests(p[ok], method="fdr_bh")
        q[ok] = q_ok
    res[pcol.replace("p_", "q_")] = q
res

,Genes,n_obs,r2,p_cell_line,p_drug,p_day,coef_drug_DAPT_vs_VEH,se_drug_DAPT_vs_VEH,t_drug_DAPT_vs_VEH,pcoef_drug_DAPT_vs_VEH,...,ci97.5_day_8_vs_3,coef_day_16_vs_3,se_day_16_vs_3,t_day_16_vs_3,pcoef_day_16_vs_3,ci2.5_day_16_vs_3,ci97.5_day_16_vs_3,q_cell_line,q_drug,q_day
0,NUDT4B,95,0.131678,2.727828e-02,0.126352,1.061714e-01,-0.091369,0.059211,-1.543111,0.126352,...,0.180928,-0.006151,0.083275,-0.073866,9.412824e-01,-0.171617,0.159315,4.424380e-02,0.420597,1.324229e-01
1,PIGBOS1,95,0.145017,3.396103e-01,0.588080,4.427435e-03,0.065144,0.119840,0.543594,0.588080,...,-0.284612,-0.367890,0.168545,-2.182732,3.168817e-02,-0.702786,-0.032993,4.094599e-01,0.829779,7.237376e-03
2,TMEM275,95,0.604732,2.954456e-15,0.004475,1.315702e-06,0.313994,0.107649,2.916825,0.004475,...,-0.212600,-0.723636,0.151400,-4.779629,6.893209e-06,-1.024465,-0.422807,2.759595e-14,0.056516,3.923346e-06
3,CENPVL1,95,0.622337,2.147679e-18,0.606685,8.725119e-05,-0.054085,0.104685,-0.516646,0.606685,...,-0.120288,-0.419606,0.147231,-2.849986,5.432085e-03,-0.712151,-0.127061,2.732048e-17,0.838838,1.943059e-04
4,NBDY,95,0.305526,1.302424e-03,0.295129,2.557654e-05,-0.117894,0.111945,-1.053137,0.295129,...,-0.504628,-0.363284,0.157443,-2.307406,2.335206e-02,-0.676119,-0.050449,2.720480e-03,0.624721,6.238737e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,MYO16,95,0.149731,1.571155e-01,0.025235,4.427061e-02,0.139129,0.061124,2.276171,0.025235,...,0.346539,0.117304,0.085966,1.364533,1.758402e-01,-0.053509,0.288117,2.119086e-01,0.164328,5.970017e-02
8748,MORC2,95,0.518826,3.192115e-11,0.406145,8.006606e-07,0.027446,0.032883,0.834658,0.406145,...,0.335284,0.243435,0.046248,5.263729,9.686282e-07,0.151542,0.335328,1.938750e-10,0.720719,2.489301e-06
8749,IVNS1ABP,95,0.471815,1.248813e-07,0.613518,3.701662e-08,-0.029279,0.057767,-0.506844,0.613518,...,-0.102307,-0.243121,0.081245,-2.992431,3.581154e-03,-0.404554,-0.081688,4.811277e-07,0.842247,1.424042e-07
8750,CAMTA1,95,0.145245,9.787987e-03,0.419735,7.044511e-02,0.067661,0.083466,0.810639,0.419735,...,0.551223,0.195441,0.117388,1.664911,9.944718e-02,-0.037807,0.428688,1.736559e-02,0.731923,9.113608e-02


In [15]:
# ===== 6) 保存结果 =====
res.to_csv(f"{file_path}/baseline_linear_model_p_qvalues.csv", index=False) 

# print(res.sort_values("q_drug").head(10))


In [16]:
res.sort_values("q_drug").head(10)

,Genes,n_obs,r2,p_cell_line,p_drug,p_day,coef_drug_DAPT_vs_VEH,se_drug_DAPT_vs_VEH,t_drug_DAPT_vs_VEH,pcoef_drug_DAPT_vs_VEH,...,ci97.5_day_8_vs_3,coef_day_16_vs_3,se_day_16_vs_3,t_day_16_vs_3,pcoef_day_16_vs_3,ci2.5_day_16_vs_3,ci97.5_day_16_vs_3,q_cell_line,q_drug,q_day
2662,PKIA,95,0.431933,7.717013e-02,4.453809e-11,4.457619e-02,0.710634,0.094691,7.504790,4.453809e-11,...,-0.111011,-0.103905,0.133175,-0.780213,4.373354e-01,-0.368521,0.160711,1.129609e-01,3.897974e-07,6.002935e-02
8090,SH3BGRL2,95,0.403570,3.775649e-01,5.538293e-10,1.384317e-02,0.375667,0.053966,6.961163,5.538293e-10,...,0.153846,0.142686,0.075899,1.879940,6.338752e-02,-0.008124,0.293495,4.480000e-01,2.423557e-06,2.061167e-02
7941,VPS29,95,0.410592,1.009105e-03,1.379265e-08,8.069461e-03,-0.135897,0.021741,-6.250781,1.379265e-08,...,0.073810,-0.086509,0.030577,-2.829249,5.765171e-03,-0.147264,-0.025754,2.144133e-03,2.011888e-05,1.253308e-02
340,ERC2,95,0.732813,8.143298e-23,1.291342e-08,3.633802e-05,0.221196,0.035303,6.265584,1.291342e-08,...,0.095976,-0.022481,0.049651,-0.452774,6.518136e-01,-0.121137,0.076175,1.608807e-21,2.011888e-05,8.646829e-05
1506,PVR,95,0.388784,1.786342e-01,1.195708e-08,2.578214e-03,0.349689,0.055658,6.282861,1.195708e-08,...,0.023129,-0.195706,0.078278,-2.500138,1.424679e-02,-0.351243,-0.040169,2.364141e-01,2.011888e-05,4.413168e-03
5274,PLA2G15,95,0.384667,1.593712e-03,1.130080e-08,9.461926e-02,0.360281,0.057228,6.295526,1.130080e-08,...,0.079979,0.081749,0.080487,1.015680,3.125350e-01,-0.078177,0.241674,3.276525e-03,2.011888e-05,1.192551e-01
878,SNCG,95,0.517077,1.710633e-09,2.407146e-08,3.527741e-03,0.460774,0.075227,6.125130,2.407146e-08,...,0.184396,0.173114,0.105801,1.636229,1.053233e-01,-0.037110,0.383338,8.410933e-09,3.009620e-05,5.873081e-03
2348,FABP6,95,0.905105,2.819798e-45,4.997734e-08,5.829435e-11,0.392097,0.065801,5.958868,4.997734e-08,...,-0.288683,-0.294010,0.092543,-3.176997,2.046010e-03,-0.477892,-0.110128,5.141433e-43,5.467521e-05,3.538087e-10
5037,CADM3,95,0.433088,1.254896e-06,5.799010e-08,8.677644e-02,0.360827,0.060901,5.924812,5.799010e-08,...,0.239731,0.179498,0.085652,2.095658,3.895240e-02,0.009309,0.349688,4.185539e-06,5.639215e-05,1.102595e-01
931,ABLIM3,95,0.574690,4.943783e-04,7.781773e-08,1.690962e-11,0.208984,0.035680,5.857235,7.781773e-08,...,0.422479,0.364464,0.050181,7.263051,1.373319e-10,0.264756,0.464172,1.113432e-03,5.784573e-05,1.139284e-10


In [17]:
import numpy as np
import pandas as pd
import warnings

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from scipy import stats

In [18]:

results = []

# baseline formula（无交互项）
FORMULA = "log2_intensity ~ C(cell_line) + C(drug) + C(day)"

# 你关心的“单个系数”（这些是相对 reference: CL2, VEH, 3d）
term_map = {
    "coef_drug_DAPT_vs_VEH": "C(drug)[T.DAPT]",
    "coef_cell_CL5_vs_CL2":  "C(cell_line)[T.CL5]",
    "coef_day_4_vs_3":       "C(day)[T.4]",
    "coef_day_8_vs_3":       "C(day)[T.8]",
    "coef_day_16_vs_3":      "C(day)[T.16]",
}

def safe_get_coef_stats(coef_tbl, row, out_name, term):
    """
    从 summary2 的 coef table 里安全读取 coef / SE / t / p / CI
    """
    if term in coef_tbl.index:
        row[out_name] = float(coef_tbl.loc[term, "Coef."])
        row[out_name.replace("coef_", "se_")] = float(coef_tbl.loc[term, "Std.Err."])
        row[out_name.replace("coef_", "t_")]  = float(coef_tbl.loc[term, "t"])
        row[out_name.replace("coef_", "pcoef_")] = float(coef_tbl.loc[term, "P>|t|"])
        row[out_name.replace("coef_", "ci2.5_")]  = float(coef_tbl.loc[term, "CI2.5%"])
        row[out_name.replace("coef_", "ci97.5_")] = float(coef_tbl.loc[term, "CI97.5%"])
    else:
        row[out_name] = np.nan
        row[out_name.replace("coef_", "se_")] = np.nan
        row[out_name.replace("coef_", "t_")]  = np.nan
        row[out_name.replace("coef_", "pcoef_")] = np.nan
        row[out_name.replace("coef_", "ci2.5_")]  = np.nan
        row[out_name.replace("coef_", "ci97.5_")] = np.nan

def add_contrast(beta_a, beta_b, var_a, var_b, cov_ab, df_resid, prefix, row):
    """
    contrast = beta_a - beta_b
    SE = sqrt(var_a + var_b - 2*cov_ab)
    t = contrast / SE
    p = 2*(1 - CDF_t(|t|))
    CI = contrast ± t_{0.975, df} * SE
    """
    coef = beta_a - beta_b
    se = np.sqrt(var_a + var_b - 2.0 * cov_ab)
    if se <= 0 or not np.isfinite(se):
        tval = pval = ci_lo = ci_hi = np.nan
    else:
        tval = coef / se
        pval = 2.0 * (1.0 - stats.t.cdf(np.abs(tval), df=df_resid))
        tcrit = stats.t.ppf(0.975, df=df_resid)
        ci_lo = coef - tcrit * se
        ci_hi = coef + tcrit * se

    row[prefix] = float(coef) if np.isfinite(coef) else np.nan
    row[prefix.replace("coef_", "se_")] = float(se) if np.isfinite(se) else np.nan
    row[prefix.replace("coef_", "t_")]  = float(tval) if np.isfinite(tval) else np.nan
    row[prefix.replace("coef_", "pcoef_")] = float(pval) if np.isfinite(pval) else np.nan
    row[prefix.replace("coef_", "ci2.5_")]  = float(ci_lo) if np.isfinite(ci_lo) else np.nan
    row[prefix.replace("coef_", "ci97.5_")] = float(ci_hi) if np.isfinite(ci_hi) else np.nan


for gene, sub in long.groupby("Genes", sort=False):
    d = sub.dropna(subset=["intensity"]).copy()
    if d.shape[0] < 10:
        continue

    d["log2_intensity"] = np.log2(d["intensity"] + 1)

    # 固定 reference（决定系数的含义）
    d["cell_line"] = pd.Categorical(d["cell_line"], categories=["CL2", "CL5"])
    d["drug"]      = pd.Categorical(d["drug"],      categories=["VEH", "DAPT"])
    d["day"]       = pd.Categorical(d["day"],       categories=[3, 4, 8, 16], ordered=True)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = smf.ols(FORMULA, data=d).fit()

    # ===== A) 因子整体显著性：Type II ANOVA (F-test) =====
    aov = sm.stats.anova_lm(model, typ=2)

    row = {
        "Genes": gene,
        "n_obs": int(d.shape[0]),
        "r2": float(model.rsquared),
        "p_cell_line": float(aov.loc["C(cell_line)", "PR(>F)"]) if "C(cell_line)" in aov.index else np.nan,
        "p_drug":      float(aov.loc["C(drug)",      "PR(>F)"]) if "C(drug)" in aov.index else np.nan,
        "p_day":       float(aov.loc["C(day)",       "PR(>F)"]) if "C(day)" in aov.index else np.nan,
    }

    # ===== B) 单个系数：coef / SE / t / p / 95%CI =====
    coef_tbl = model.summary2().tables[1].copy()  # Coef., Std.Err., t, P>|t|
    ci = model.conf_int(alpha=0.05)
    ci.columns = ["CI2.5%", "CI97.5%"]
    coef_tbl = coef_tbl.join(ci)

    for out_name, term in term_map.items():
        safe_get_coef_stats(coef_tbl, row, out_name, term)

    # ===== C) 额外 contrasts：8 vs 4, 16 vs 4（同一模型内线性组合）=====
    params = model.params
    cov = model.cov_params()
    df_resid = model.df_resid

    # 8 vs 4
    if ("C(day)[T.8]" in params) and ("C(day)[T.4]" in params):
        add_contrast(
            beta_a=params["C(day)[T.8]"],
            beta_b=params["C(day)[T.4]"],
            var_a=cov.loc["C(day)[T.8]", "C(day)[T.8]"],
            var_b=cov.loc["C(day)[T.4]", "C(day)[T.4]"],
            cov_ab=cov.loc["C(day)[T.8]", "C(day)[T.4]"],
            df_resid=df_resid,
            prefix="coef_day_8_vs_4",
            row=row
        )
    else:
        # 填空列保持一致
        for k in ["coef_day_8_vs_4","se_day_8_vs_4","t_day_8_vs_4","pcoef_day_8_vs_4","ci2.5_day_8_vs_4","ci97.5_day_8_vs_4"]:
            row[k] = np.nan

    # 16 vs 4
    if ("C(day)[T.16]" in params) and ("C(day)[T.4]" in params):
        add_contrast(
            beta_a=params["C(day)[T.16]"],
            beta_b=params["C(day)[T.4]"],
            var_a=cov.loc["C(day)[T.16]", "C(day)[T.16]"],
            var_b=cov.loc["C(day)[T.4]",  "C(day)[T.4]"],
            cov_ab=cov.loc["C(day)[T.16]", "C(day)[T.4]"],
            df_resid=df_resid,
            prefix="coef_day_16_vs_4",
            row=row
        )
    else:
        for k in ["coef_day_16_vs_4","se_day_16_vs_4","t_day_16_vs_4","pcoef_day_16_vs_4","ci2.5_day_16_vs_4","ci97.5_day_16_vs_4"]:
            row[k] = np.nan

    results.append(row)

res = pd.DataFrame(results)

# ===== D) 对“因子整体P值”做 BH-FDR（q值）=====
for pcol in ["p_cell_line", "p_drug", "p_day"]:
    p = res[pcol].astype(float).values
    ok = np.isfinite(p)
    q = np.full_like(p, np.nan, dtype=float)
    if ok.sum() > 0:
        _, q_ok, _, _ = multipletests(p[ok], method="fdr_bh")
        q[ok] = q_ok
    res[pcol.replace("p_", "q_")] = q

res


,Genes,n_obs,r2,p_cell_line,p_drug,p_day,coef_drug_DAPT_vs_VEH,se_drug_DAPT_vs_VEH,t_drug_DAPT_vs_VEH,pcoef_drug_DAPT_vs_VEH,...,ci97.5_day_8_vs_4,coef_day_16_vs_4,se_day_16_vs_4,t_day_16_vs_4,pcoef_day_16_vs_4,ci2.5_day_16_vs_4,ci97.5_day_16_vs_4,q_cell_line,q_drug,q_day
0,NUDT4B,95,0.131678,2.727828e-02,0.126352,1.061714e-01,-0.091369,0.059211,-1.543111,0.126352,...,0.348688,0.161609,0.083275,1.940662,0.055464,-0.003857,0.327075,4.424380e-02,0.420597,1.324229e-01
1,PIGBOS1,95,0.145017,3.396103e-01,0.588080,4.427435e-03,0.065144,0.119840,0.543594,0.588080,...,0.135908,0.052630,0.168545,0.312263,0.755571,-0.282266,0.387527,4.094599e-01,0.829779,7.237376e-03
2,TMEM275,95,0.604732,2.954456e-15,0.004475,1.315702e-06,0.313994,0.107649,2.916825,0.004475,...,-0.218476,-0.729513,0.151400,-4.818443,0.000006,-1.030341,-0.428684,2.759595e-14,0.056516,3.923346e-06
3,CENPVL1,95,0.622337,2.147679e-18,0.606685,8.725119e-05,-0.054085,0.104685,-0.516646,0.606685,...,0.596649,0.297331,0.147231,2.019485,0.046445,0.004786,0.589876,2.732048e-17,0.838838,1.943059e-04
4,NBDY,95,0.305526,1.302424e-03,0.295129,2.557654e-05,-0.117894,0.111945,-1.053137,0.295129,...,-0.218980,-0.077636,0.157443,-0.493106,0.623152,-0.390471,0.235199,2.720480e-03,0.624721,6.238737e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8747,MYO16,95,0.149731,1.571155e-01,0.025235,4.427061e-02,0.139129,0.061124,2.276171,0.025235,...,0.105995,-0.123240,0.085966,-1.433585,0.155195,-0.294053,0.047573,2.119086e-01,0.164328,5.970017e-02
8748,MORC2,95,0.518826,3.192115e-11,0.406145,8.006606e-07,0.027446,0.032883,0.834658,0.406145,...,0.201253,0.109404,0.046248,2.365617,0.020171,0.017511,0.201297,1.938750e-10,0.720719,2.489301e-06
8749,IVNS1ABP,95,0.471815,1.248813e-07,0.613518,3.701662e-08,-0.029279,0.057767,-0.506844,0.613518,...,0.448945,0.308131,0.081245,3.792595,0.000271,0.146698,0.469564,4.811277e-07,0.842247,1.424042e-07
8750,CAMTA1,95,0.145245,9.787987e-03,0.419735,7.044511e-02,0.067661,0.083466,0.810639,0.419735,...,0.358449,0.002666,0.117388,0.022713,0.981930,-0.230581,0.235914,1.736559e-02,0.731923,9.113608e-02


In [19]:
# ===== 6) 保存结果 =====
res.to_csv(f"{file_path}/baseline_linear_model_p_qvalues2.csv", index=False) 

In [20]:
res.columns


Index(['Genes', 'n_obs', 'r2', 'p_cell_line', 'p_drug', 'p_day',
       'coef_drug_DAPT_vs_VEH', 'se_drug_DAPT_vs_VEH', 't_drug_DAPT_vs_VEH',
       'pcoef_drug_DAPT_vs_VEH', 'ci2.5_drug_DAPT_vs_VEH',
       'ci97.5_drug_DAPT_vs_VEH', 'coef_cell_CL5_vs_CL2', 'se_cell_CL5_vs_CL2',
       't_cell_CL5_vs_CL2', 'pcoef_cell_CL5_vs_CL2', 'ci2.5_cell_CL5_vs_CL2',
       'ci97.5_cell_CL5_vs_CL2', 'coef_day_4_vs_3', 'se_day_4_vs_3',
       't_day_4_vs_3', 'pcoef_day_4_vs_3', 'ci2.5_day_4_vs_3',
       'ci97.5_day_4_vs_3', 'coef_day_8_vs_3', 'se_day_8_vs_3', 't_day_8_vs_3',
       'pcoef_day_8_vs_3', 'ci2.5_day_8_vs_3', 'ci97.5_day_8_vs_3',
       'coef_day_16_vs_3', 'se_day_16_vs_3', 't_day_16_vs_3',
       'pcoef_day_16_vs_3', 'ci2.5_day_16_vs_3', 'ci97.5_day_16_vs_3',
       'coef_day_8_vs_4', 'se_day_8_vs_4', 't_day_8_vs_4', 'pcoef_day_8_vs_4',
       'ci2.5_day_8_vs_4', 'ci97.5_day_8_vs_4', 'coef_day_16_vs_4',
       'se_day_16_vs_4', 't_day_16_vs_4', 'pcoef_day_16_vs_4',
       'ci2.5_day_

In [7]:
exam = pd.read_csv(f"{file_path}/baseline_linear_model_p_qvalues2.csv",index_col = 0 ) 

# Visualization 